In [2]:
!pip install nest_asyncio

'pip' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [18]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
uvicorn.run(app, host="localhost", port=8000)

INFO:     Started server process [22276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:58794 - "GET / HTTP/1.1" 200 OK
INFO:     ::1:58795 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:58795 - "GET /openapi.json HTTP/1.1" 200 OK


c:\Users\sugiyamase\AppData\Local\Programs\Python\Python312\Lib\site-packages\fastapi\openapi\utils.py:246: UserWarning: Duplicate Operation ID get_product_api_product__code__get for function get_product
  warnings.warn(message, stacklevel=1)
c:\Users\sugiyamase\AppData\Local\Programs\Python\Python312\Lib\site-packages\fastapi\openapi\utils.py:246: UserWarning: Duplicate Operation ID purchase_api_purchase_post for function purchase
  warnings.warn(message, stacklevel=1)


INFO:     127.0.0.1:58836 - "GET /api/product/4901777300446 HTTP/1.1" 200 OK
INFO:     127.0.0.1:58836 - "GET /api/product/4901777300446 HTTP/1.1" 200 OK
INFO:     127.0.0.1:58853 - "GET /api/product/4901777300446 HTTP/1.1" 200 OK
INFO:     127.0.0.1:59433 - "GET /api/product/4901777300446 HTTP/1.1" 200 OK
INFO:     127.0.0.1:59449 - "GET /api/product/4901777300446 HTTP/1.1" 200 OK
INFO:     127.0.0.1:59460 - "GET /api/product/4901111000003 HTTP/1.1" 200 OK
INFO:     127.0.0.1:59460 - "POST /api/purchase HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22276]


In [5]:
pip install fastapi uvicorn pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ライブラリのインポート

In [7]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional
import pymysql
from datetime import datetime

FastAPIアプリとCORSの設定

In [8]:
app = FastAPI()

# CORSの設定
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 本番環境では適切に制限すること
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

データベース接続関数

In [9]:
# データベース接続設定
def get_db_connection():
    return pymysql.connect(
        host="localhost",
        user="root",
        password="mfM236#:",  # MySQLのrootパスワードを入力
        database="pos_app",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )

モデル定義

In [12]:
# モデル定義
class Product(BaseModel):
    prd_id: Optional[int] = None
    code: str
    name: str
    price: int

class PurchaseItem(BaseModel):
    prd_id: int
    code: str
    name: str
    price: int
    quantity: int = 1

class PurchaseRequest(BaseModel):
    emp_cd: str = "9999999999"
    store_cd: str = "30"
    pos_id: str = "90"
    items: List[PurchaseItem]

class PurchaseResponse(BaseModel):
    success: bool
    total_amount: int

APIエンドポイント - ルート

In [13]:
@app.get("/")
def read_root():
    return {"message": "Welcome to POS API"}

APIエンドポイント - 商品取得

In [15]:
@app.get("/api/product/{code}")
def get_product(code: str):
    conn = get_db_connection()
    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT PRD_ID, CODE, NAME, PRICE FROM products WHERE CODE = %s", (code,))
            product = cursor.fetchone()
            if not product:
                return {"product": None}
            return {
                "product": {
                    "prd_id": product["PRD_ID"],
                    "code": product["CODE"].strip(),  # CHAR型のため、空白を削除
                    "name": product["NAME"],
                    "price": product["PRICE"]
                }
            }
    finally:
        conn.close()

APIエンドポイント - 購入処理

In [17]:
@app.post("/api/purchase")
def purchase(request: PurchaseRequest):
    conn = get_db_connection()
    try:
        with conn.cursor() as cursor:
            # 1. 取引テーブルに登録
            cursor.execute(
                "INSERT INTO transactions (DATETIME, EMP_CD, STORE_CD, POS_NO, TOTAL_AMT) VALUES (%s, %s, %s, %s, %s)",
                (datetime.now(), request.emp_cd, request.store_cd, request.pos_id, 0)
            )
            trd_id = cursor.lastrowid
            conn.commit()
            
            # 2. 取引明細テーブルに登録
            total_amount = 0
            for idx, item in enumerate(request.items, 1):
                cursor.execute(
                    "INSERT INTO transaction_details (TRD_ID, DTL_ID, PRD_ID, PRD_CODE, PRD_NAME, PRD_PRICE) VALUES (%s, %s, %s, %s, %s, %s)",
                    (trd_id, idx, item.prd_id, item.code, item.name, item.price)
                )
                total_amount += item.price * item.quantity
            
            # 3. 取引テーブルの合計金額を更新
            cursor.execute(
                "UPDATE transactions SET TOTAL_AMT = %s WHERE TRD_ID = %s",
                (total_amount, trd_id)
            )
            conn.commit()
            
            return {"success": True, "total_amount": total_amount}
    except Exception as e:
        conn.rollback()
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        conn.close()